# LSTM Text Classifier
## Sentiment Analysis

In [19]:
import sys
print(sys.prefix)
!which python

/Users/eunbeejang/anaconda3
/usr/local/bin/python


In [75]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe, Vectors
from torch.autograd import Variable
from torch.nn import functional

import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()


In [82]:
def load_data():
    tokenize = lambda x: lemma.lemmatize(re.sub(r'<.*?>|[^\w\s]|\d+', '', x)).split()
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True,
                      fix_length=200, include_lengths=True, batch_first=True)
    LABEL = data.Field(sequential=False)
    
    train, test = datasets.IMDB.splits(TEXT, LABEL)
    TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300))
    LABEL.build_vocab(train)
    word_emb = TEXT.vocab.vectors
    
    print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
    print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
    print ("Label Length: " + str(len(LABEL.vocab)))
    train, valid = train.split()
    train_data, valid_data, test_data = data.BucketIterator.splits((train, valid, test),
                                                                   batch_size=32, repeat=False, shuffle=True)

    vocab_size = len(TEXT.vocab)

    return TEXT, word_emb, train_data, valid_data, test_data, vocab_size

In [83]:
TEXT, word_emb, train_data, valid_data, test_data, vocab_size = load_data()

Loading vectors from .vector_cache/glove.6B.300d.txt.pt


Length of Text Vocabulary: 138163
Vector size of Text Vocabulary:  torch.Size([138163, 300])
Label Length: 3


In [86]:
word_emb.size()

torch.Size([138163, 300])

In [87]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 328121), ('and', 161575), ('a', 161309), ('of', 145166), ('to', 134822), ('is', 106799), ('in', 92187), ('it', 76313), ('this', 73186), ('i', 72475), ('that', 69198), ('was', 47988), ('as', 46058), ('with', 43724), ('for', 43701), ('movie', 41826), ('but', 40999), ('film', 37487), ('on', 33340), ('not', 30012)]


In [88]:
print(TEXT.vocab.itos[:20])

['<unk>', '<pad>', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'this', 'i', 'that', 'was', 'as', 'with', 'for', 'movie', 'but', 'film']


In [94]:
class LSTM(torch.nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim, embedding, batch_size):
        super(LSTM, self).__init__()
        
        self.input_dim = input_dim # vocabulary dim
        self.emb_dim = emb_dim # glove dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.embedding = embedding # glove embedding
        self.batch_size = batch_size
        
        # Initalize look-up table
        self.word_emb = torch.nn.Embedding(input_dim, emb_dim)
        # connect look-up table to glove embedding
        self.word_emb.weight = torch.nn.Parameter(embedding, requires_grad=False)
        
        # Layers: one LSTM, one Fully-connected
        self.lstm = nn.LSTM(emb_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # input_emb (num_seq, batch_size, emb_dim)
        emb_input = self.word_emb(x).permute(1, 0, 2)
        print(emb_input.shape)
        hidden_state = Variable(torch.zeros(1, self.batch_size, self.hidden_size))
        cell_state = Variable(torch.zeros(1, self.batch_size, self.hidden_size))
        # hidden_out, cell_out shape: (1, batch_size, hidden_dim)
        _, (hidden_out, cell_out) = self.lstm(emb_input, (hidden_state, cell_state))
        print("hidden shape: ", hidden_out.shape)
        y_pred = self.fc(hidden_out[-1])

        return y_pred # (batch_size, output_dim)

In [101]:
input_dim = vocab_size
emb_dim = 300
hidden_dim = 256
output_dim = 2
embedding = word_emb
batch_size = 32

model = LSTM(input_dim, emb_dim, hidden_dim, output_dim, embedding, batch_size)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(filter(lambda param: param.requires_grad, model.parameters()),lr=0.01)

In [98]:
model.parameters()

<generator object Module.parameters at 0x1a2d5bbc50>

In [102]:
def train(model, data_iter, epoch_size, criterion, optimizer):
    
    model.train()

    for epoch in range(epoch_size):    
        #for batch in enumerate(data_iter):
        
            pred = model(batch.text).squeeze(1)
            loss = criterion(pred, batch.label)
            correct = (pred == batch.label).float().sum()

            optimizer.zero_grad()      
            loss.backward()
            optimizer.step()
        
           #print("Epoch {}/{}, Loss: {:.4f}, Accuracy: {:.4f}".format(epoch+1,epoch_size, loss.data[0], correct/pred.shape[0]))

        

In [63]:
#train(model, train_iter, 20, criterion, optimizer)

NameError: name 'model' is not defined